In [8]:

import pandas as pd

df = pd.read_csv('/content/Carseats.csv')
print(df)


     Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0     9.50        138      73           11         276    120       Bad   42   
1    11.22        111      48           16         260     83      Good   65   
2    10.06        113      35           10         269     80    Medium   59   
3     7.40        117     100            4         466     97    Medium   55   
4     4.15        141      64            3         340    128       Bad   38   
..     ...        ...     ...          ...         ...    ...       ...  ...   
395  12.57        138     108           17         203    128      Good   33   
396   6.14        139      23            3          37    120    Medium   55   
397   7.41        162      26           12         368    159    Medium   40   
398   5.94        100      79            7         284     95       Bad   50   
399   9.71        134      37            0          27    120      Good   49   

     Education Urban   US  
0          

In [9]:
# Encoding Urban and US to numerical

from sklearn.preprocessing import LabelEncoder

df.dropna(inplace=True)
le = LabelEncoder()
df['Urban'] = le.fit_transform(df['Urban'])
df['US'] = le.fit_transform(df['US'])
print(df)


     Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0     9.50        138      73           11         276    120       Bad   42   
1    11.22        111      48           16         260     83      Good   65   
2    10.06        113      35           10         269     80    Medium   59   
3     7.40        117     100            4         466     97    Medium   55   
4     4.15        141      64            3         340    128       Bad   38   
..     ...        ...     ...          ...         ...    ...       ...  ...   
395  12.57        138     108           17         203    128      Good   33   
396   6.14        139      23            3          37    120    Medium   55   
397   7.41        162      26           12         368    159    Medium   40   
398   5.94        100      79            7         284     95       Bad   50   
399   9.71        134      37            0          27    120      Good   49   

     Education  Urban  US  
0          

This question should be answered using the Carseats data set.

a) Fit a multiple regression model to predict Sales using Price, Urban, and US.

In [10]:

from sklearn import linear_model
from sklearn.model_selection import train_test_split

# Declare the predictors and response
X = df[['Price', 'Urban', 'US']]
y = df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the multiple regression model
model = linear_model.LinearRegression()

model.fit(X_train, y_train)



LinearRegression()

b) Provide an interpretation of each coefficient in the model. Be careful—some of the variables in the model are qualitative!

In [11]:

# Print the coefficients
coefficients = model.coef_

# Create a DataFrame to display coefficients with corresponding features
feature_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': coefficients})
feature_importance = feature_importance.reindex(feature_importance['Coefficient'].abs().sort_values(ascending=False).index)
print(feature_importance)

print('Intercept ' + str(model.intercept_))


  Feature  Coefficient
2      US     1.097797
1   Urban    -0.109045
0   Price    -0.052808
Intercept 12.900230314483789


c) Write out the model in equation form, being careful to handle the qualitative variables properly.

y^ =

     itrcp + (b1*X['Price']) + b2 + b3 , when "Urban" = Yes and "US" = Yes

     itrcp + (b1*X['Price']) + b2      , when "Urban" = Yes and "US" = No

     itrcp + (b1*X['Price']) + b3      , when "Urban" = No and "US" = Yes

     itrcp + (b1*X['Price'])           , when "Urban" = No and "US" = No

,where

       itrcp = intercept of the model (b0)

       b1 = coefficient of the "Price" predictor

       b2 = coefficient of the "Urban" predictor

       b3 = coefficient of the "US" predictor

d) For which of the predictors can you reject the null hypothesis $H_0 : \beta_j = 0$?

Only the predictors "Urban" and "US" have coefficients $\beta_2$ and $\beta_3$ significantly different from zero while the coefficient of the other predictor $\beta_1$ is closer to zero. So, only the predictors "Urban" and "US" seems to reject the given null hypothesis.

e) On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.

In [13]:
# Declare the predictors and response for the new model
X_new = df[['Urban', 'US']]
y_new = df['Sales']

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# Fit the multiple regression model
new_model = linear_model.LinearRegression()

new_model.fit(X_train_new, y_train_new)


LinearRegression()

f) How well do the models in (a) and (e) ft the data?

In [14]:
from sklearn.metrics import r2_score

acc_a = model.score(X_test, y_test)
print(f"Test Accuracy of the model in (a): {acc_a}")

acc_e = new_model.score(X_test_new, y_test_new)
print(f"Test Accuracy of the model in (e): {acc_e}")


Test Accuracy of the model in (a): 0.21433892437588353
Test Accuracy of the model in (e): 0.002849228655444125


Based on the

f) Using the model from (e), obtain 95 % confidence intervals for the coefficient(s)

In [15]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from scipy import stats

# ... (previous code from the prompt) ...

# Get the coefficients and standard errors
coef = new_model.coef_
X_new = df[['Urban', 'US']]
y_new = df['Sales']
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)
n = len(y_train_new)
p = X_train_new.shape[1]
residuals = y_train_new - new_model.predict(X_train_new)
mse = np.sum(residuals**2) / (n - p - 1)
var_covar_matrix = np.linalg.inv(np.dot(X_train_new.T, X_train_new)) * mse
std_err = np.sqrt(np.diag(var_covar_matrix))

# Calculate the confidence intervals
t_critical = stats.t.ppf(1 - 0.025, df=n - p - 1)  # For 95% confidence interval
conf_intervals = [(coef[i] - t_critical * std_err[i], coef[i] + t_critical * std_err[i]) for i in range(len(coef))]

# Print the confidence intervals
print("Confidence Intervals for Coefficients:")
for i, interval in enumerate(conf_intervals):
  print(f"Coefficient {i+1}: {interval}")


Confidence Intervals for Coefficients:
Coefficient 1: (-0.7374621325797853, 0.23342732865821647)
Coefficient 2: (0.4827739781556547, 1.5026462330057337)


h) Is there evidence of outliers or high leverage observations in the model from (e)?

No, there is no evidence of outliers and high leverage observations in the model from (e).